In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('/content/hr_job_change.csv')
X = dataset.drop(columns=['target'])
y = dataset['target']

In [3]:
X = pd.get_dummies(X)

In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [5]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaler = scaler.fit(x_train)
x_train = X_scaler.fit_transform(x_train)
x_test = X_scaler.transform(x_test)

In [6]:
import lightgbm as lgb
d_train = lgb.Dataset(x_train, label=y_train)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
clf = lgb.train(params, d_train, 100)

[LightGBM] [Info] Number of positive: 3321, number of negative: 10189
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009158 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 904
[LightGBM] [Info] Number of data points in the train set: 13510, number of used features: 109
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.245818 -> initscore=-1.121043
[LightGBM] [Info] Start training from score -1.121043


In [7]:
#Prediction
y_pred=clf.predict(x_test)
#convert into binary values
for i in range(0,99):
    if y_pred[i]>=.5:       # setting threshold to .5
       y_pred[i]=1
    else:  
       y_pred[i]=0

In [8]:
y_pred = y_pred.round(0)
y_test = y_test.round(0)

In [9]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[3404,    0],
       [1100,    0]])

In [10]:
#Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)
print(f'Accuracy score: {accuracy:.2f}')

Accuracy score: 0.76


In [11]:
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [12]:
# Code is sourced from https://machinelearningmastery.com/how-to-configure-k-fold-cross-validation/

# Prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)

# Create model
model = lightgbm()

# Evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# Report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

NameError: ignored